In [ ]:
def condition_by_t(x, y):
    """Условия при t=0"""
    return np.cos(3 * x) * np.sin(4 * y)


def condition_by_y(U):
    """Условия при y=0 и y=pi/2"""
    U[:, 0] = 0
    U[:, -1] = 0
    return U


def condition_by_x(U):
    """Условия при du/dx=0 при x=0 и du/dx=0 при x=pi/3"""
    U[0, :] = U[1, :]
    U[nx, :] = U[nx - 1, :]
    return U

In [4]:
import numpy as np
import matplotlib.pyplot as plt

# Параметры задачи
Lx = np.pi
Ly = np.pi
T = 0.5  # конечное время

Nx = 50  # число шагов по x
Ny = 50  # число шагов по y
Nt = 100  # число шагов по времени

dx = Lx / (Nx - 1)
dy = Ly / (Ny - 1)
dt = T / Nt

# Коэффициенты для схемы
alpha_x = dt / (dx ** 2)
alpha_y = dt / (dy ** 2)

# Пространственные сетки
x = np.linspace(0, Lx, Nx)
y = np.linspace(0, Ly, Ny)
t = np.linspace(0, T, Nt)

# Начальные условия
u = np.zeros((Nx, Ny))

# u(x, y, 0) = sin(x) * (1 + 2 * y * cos(x))
for i in range(Nx):
    for j in range(Ny):
        u[i, j] = np.cos(3 * i) * np.sin(4 * j)


# Исправление индексации в методе прогонки
def solve_tridiagonal(a, b, c, d):
    n = len(d)
    c_new = np.zeros(n-1)
    d_new = np.zeros(n)
    x = np.zeros(n)

    # Прямой ход прогонки
    c_new[0] = c[0] / b[0]
    d_new[0] = d[0] / b[0]
    
    for i in range(1, n-1):
        denom = b[i] - a[i-1] * c_new[i-1]
        c_new[i] = c[i] / denom
        d_new[i] = (d[i] - a[i-1] * d_new[i-1]) / denom

    # Обратный ход прогонки
    d_new[-1] = (d[-1] - a[-1] * d_new[-2]) / (b[-1] - a[-1] * c_new[-2])
    x[-1] = d_new[-1]
    for i in range(n-2, -1, -1):
        x[i] = d_new[i] - c_new[i] * x[i+1]
    
    return x

# Основной цикл по времени
for n in range(1, Nt):
    # Шаг 1: Решаем по x при фиксированном y
    u_temp = np.copy(u)
    for j in range(1, Ny-1):
        a = -alpha_x * np.ones(Nx-1)
        b = (1 + 2 * alpha_x) * np.ones(Nx)
        c = -alpha_x * np.ones(Nx-1)
        d = u_temp[:, j]


        u[:, j] = solve_tridiagonal(a, b, c, d)
    u[0, :] = u[1, :]
    u[Nx, :] = u[Nx - 1, :]

    # Шаг 2: Решаем по y при фиксированном x
    u_temp = np.copy(u)
    for i in range(1, Nx-1):
        a = -alpha_y * np.ones(Ny-1)
        b = (1 + 2 * alpha_y) * np.ones(Ny)
        c = -alpha_y * np.ones(Ny-1)
        d = u_temp[i, :]

        # Граничные условия для y

        u[i, :] = solve_tridiagonal(a, b, c, d)
    u[:, 0] = 0
    u[:, -1] = 0

# Построение графика решения для фиксированного времени
X, Y = np.meshgrid(x, y)
plt.figure(figsize=(8, 6))
plt.contourf(X, Y, u.T, 20)
plt.colorbar(label='u(x, y)')
plt.title('Решение уравнения теплопроводности для фиксированного времени')
plt.xlabel('x')
plt.ylabel('y')
plt.show()

u


IndexError: index 50 is out of bounds for axis 0 with size 50